In [1]:
import pandas as pd
import numpy as np
training_df = pd.read_csv('./data/cleaned_Training_set_values.csv', index_col='id')
training_label_df = pd.read_csv('./data/Training_set_labels.csv', index_col='id')
training_cols = list(training_df.columns.values)
for col in training_cols:
    if training_df[col].dtype == np.object:
        training_df[col] = training_df[col].astype('category')
training_df['district_code'] = training_df['district_code'].astype('category')
training_df['region_code'] = training_df['region_code'].astype('category')
print(training_df.dtypes)

amount_tsh                float64
gps_height                float64
basin                    category
subvillage               category
region_code              category
district_code            category
lga                      category
ward                     category
population                float64
public_meeting            float64
scheme_management        category
permit                    float64
construction_year         float64
extraction_type          category
extraction_type_group    category
extraction_type_class    category
management               category
management_group         category
payment                  category
payment_type             category
water_quality            category
quality_group            category
quantity                 category
quantity_group           category
source                   category
source_type              category
source_class             category
waterpoint_type          category
waterpoint_type_group    category
dtype: object


In [2]:
vect_df = pd.get_dummies(training_df,  sparse=True)
vect_df.to_pickle('./data/clean_vect_training.pickle')

In [5]:
import pandas as pd
import numpy as np
vect_df = pd.read_pickle('./data/vect_training.pickle')
training_label_df = pd.read_csv('./data/Training_set_labels.csv', index_col='id')

In [6]:
repl_val = ['functional', 'functional needs repair', 'non functional']
y_df = training_label_df[['status_group']]
y_df = y_df.replace(repl_val, [0.0, 1.0, 2.0])
y = y_df.as_matrix().astype(np.float)

In [3]:
print(vect_df)

NameError: name 'vect_df' is not defined

In [2]:
X = vect_df.as_matrix().astype(np.float)

# This is important
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
X = Imputer().fit_transform(X).StandardScaler().fit_transform(X)

In [1]:
print("Feature space holds %d observations and %d features" % X.shape)
print("Unique target labels:", np.unique(y))

In [ ]:
from sklearn.cross_validation import KFold


def run_cv(X, y, clf_class, **kwargs):
    # Construct a kfolds object
    kf = KFold(len(y),n_folds=5,shuffle=True)
    y_pred = y.copy()

    # Iterate through folds
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        # Initialize a classifier with key word arguments
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN

def accuracy(y_true,y_pred):
    # NumPy interprets True and False as 1. and 0.
    return np.mean(y_true == y_pred)

In [ ]:
print("Support vector machines:")
print("%.3f" % accuracy(y, run_cv(X,y,SVC)))

In [ ]:
print("Random forest:")
print("%.3f" % accuracy(y, run_cv(X,y,RF)))

In [ ]:
print("K-nearest-neighbors:")
print("%.3f" % accuracy(y, run_cv(X,y,KNN)))